In [2]:
from random import randrange, random
from random import choices
from heapq import nlargest
from statistics import mean
from copy import deepcopy

### Инициализируем первичную популяцию - 20 случайных наборов чисел вида (x,y):

In [3]:
initial_population = [[randrange(100), randrange(100)] for _ in range(20)]

### Наша задача: найти такой **`item (x,y)`**, при которых максимизируется функция **`y = 6*x - x*x+4*y - y*y`** - функция **`ВЫЖИВАНИЯ`**:

In [4]:
def fitness(item):
    return 6 * item[0] - item[0] * item[0] + 4 * item[1] - item[1] * item[1]

### Устанавливаем две функции, которые будут делать **`ОТБОР ПО ВЫЖИВАЕМОСТИ`** наиболее приспособленных (то есть тех, у кого на данный момент наилучший показатель функции **`fitness`** ). Возвращает по 2 экземпляра из первоначальной популяции с вероятностью, пропорциональной приспособленности:

In [5]:
def pick_roulette(population, wheel):
    return tuple(choices(population, weights=wheel, k=2))

In [6]:
def pick_tournament(population, num_participants):
    participants = choices(population, k=num_participants)
    return tuple(nlargest(2, participants, key=fitness))

### Реализуем алгоритм **`СКРЕЩИВАНИЯ`** - когда вышеупомянутые два экземпляра при превышении определенной вероятности рождают потомков с измененными свойствами:

In [7]:
def crossover(item1, item2):
    return [item1[0], item2[1]], [item2[0], item1[1]]

### Вводим функцию **`МУТАЦИИ`** - в отличие от скрещивания, экземпляр сам меняет свои свойства с определенной вероятностью:

In [8]:
def mutate(item_):
    item = deepcopy(item_)
    if random() > 0.5: # mutate x
        if random() > 0.5:
            item[0] += 1
        else:
            item[0] -= 1
    else: # otherwise mutate y
        if random() > 0.5:
            item[1] += 1
        else:
            item[1] -= 1
    return item

### Собираем воедино функции отбора по выживаемости, скрещивания. Возвращаем измененную популяцию:

In [9]:
def reproduce_and_replace(population):
    new_population = []
    while len(new_population) < len(population):

        if random() > 0.5:
            parents = pick_roulette(population, wheel=[fitness(item) for item in population])
        else:
            parents = pick_tournament(population, len(population) // 2)

        if random() < 0.7:
            new_population.extend(crossover(parents[0], parents[1]))
        else:
            new_population.extend(parents)

    if len(new_population) > len(population):
        new_population.pop()
    return new_population

### Собираем все воедино в функцию запуска, где устанавливаем 100 итераций-прогонов, в ходе которых популяция обновляется путем отбора, скрещивания и мутаций. Отбор происходит случайно, но предпочтение отдается экземплярам, наиболее подходящих под максимизацию функции выживаемости, в итоге выбирается наилучший экземпляр наиболее максимизирующий целевую функцию:

In [10]:
def run(population_):
    population = deepcopy(population_)
    best= max(population, key=fitness)
    for generation in range(100):
        # early exit if we beat threshold
        if fitness(best) >= 13:
            info = "X: {0} Y: {1} Fitness: {2}".format(best[0], best[1],fitness(best))
            return print(info)
        print(f"Generation {generation} Best {fitness(best)} Avg {mean(map(fitness, population))}")
        population = reproduce_and_replace(population)
        population = [mutate(item) for item in population]

        highest = max(population, key=fitness)
        if fitness(highest) > fitness(best):
            best = highest # found a new best
    #return best # best we found in _max_generations
    info = "X: {0} Y: {1} Fitness: {2}".format(best[0], best[1],fitness(best))
    return print(info)

In [11]:
run(initial_population)

Generation 0 Best -667 Avg -6417.4
Generation 1 Best -536 Avg -2037.35
Generation 2 Best -432 Avg -2811.65
Generation 3 Best -428 Avg -560.2
Generation 4 Best -428 Avg -561.15
Generation 5 Best -396 Avg -518.55
Generation 6 Best -357 Avg -433.2
Generation 7 Best -320 Avg -380.4
Generation 8 Best -285 Avg -356.55
Generation 9 Best -252 Avg -324.65
Generation 10 Best -216 Avg -321.4
Generation 11 Best -184 Avg -231.75
Generation 12 Best -157 Avg -220.1
Generation 13 Best -132 Avg -192.45
Generation 14 Best -131 Avg -195.4
Generation 15 Best -109 Avg -167.15
Generation 16 Best -88 Avg -129.95
Generation 17 Best -69 Avg -130.9
Generation 18 Best -68 Avg -118.45
Generation 19 Best -51 Avg -88.8
Generation 20 Best -36 Avg -73.2
Generation 21 Best -36 Avg -77.9
Generation 22 Best -36 Avg -82.35
Generation 23 Best -32 Avg -70.75
Generation 24 Best -16 Avg -50.25
Generation 25 Best -7 Avg -26.7
Generation 26 Best 3 Avg -33.35
Generation 27 Best 3 Avg -11.55
Generation 28 Best 5 Avg -11.5
Genera

### Математически, да и в целом с точки зрения машинного обучения, первоначальная популяция - это случайный набор чисел в соотв-щем векторном пространстве. И мы по сути решаем задачу машинного обучения - найти подпространство, такой участок, где точки будут соотв-вать обозначенным критериям (максимизации, классификации, чего угодно). И последовательный отбор, скрещивания и мутации - это плавное перемещение точек в этом векторном пространстве, эвристически стараясь не упустить может быть тонкую лазейку в этот искомый участок пространства. Все как обычно, как будто градиентный спуск, просто движения точек стохастичны и не застревают в локальных минимумах, как я понял.

### Огромный плюс алгоритма - в универсальности, нет нужды думать над прямым аналитическим выводом и т.п. Говорят он еще как применим, будем исследовать, судя по всему этот алгоритм очень перспективен. Главное он удобен. Можно исследовать все что угодно, нет нужды собирать датасеты/таргеты/функции потерь и т.п.